In [ ]:
# !unzip /content/final_casia.zip

In [ ]:
from IPython.display import JSON
from google.colab import output
# from subprocess import getoutput
from future.moves.subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest_data.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [30]:
import cv2
import numpy as np
import glob
import pickle
from sklearn.externals import joblib
from keras.applications.vgg16 import preprocess_input
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import glob
import keras
from keras.models       import Model
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121
from keras.models import Model
import glob

In [78]:
my_images = []

my_img_vals = []

lables = []

for flp in glob.iglob('processed_data/*'):
  
    if flp[-1] == 'g':
        
        img	= cv2.imread(flp)
        my_images_colored = cv2.imread(flp)
        split = flp.split(".")
        label = split[0][15:]
        image_numb = split[1]
        my_images.append([my_images_colored,int(label)])
        

    
import random

random.shuffle(my_images)

for i,j in my_images:
        my_img_vals.append(i)
        lables.append(j)
    
print(len(my_img_vals))
print(len(lables))



404
404


In [79]:
img_size = (150,200)

In [80]:
y = keras.utils.to_categorical(lables, num_classes=42, dtype='float32')
my_img_vals = np.array(my_img_vals,dtype="float16")/255

In [81]:
from keras.models       import Model
from keras.applications.densenet import DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception

#my_model = VGG16(include_top=False)
my_model = DenseNet201(include_top=False)
#my_model = InceptionV3(include_top=False)
#my_model = InceptionResNetV2(include_top=False)
#my_model = Xception(include_top=False)

model_IP  = my_model.get_layer(index=0).input
model_OP = my_model.get_layer(index=-59).output
model_OP_fin  = Model(inputs=model_IP,  outputs=model_OP)

model_OP_fin.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d_6 (ZeroPadding2D (None, None, None, 3 0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_6[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [82]:
print(len(model_OP_fin.layers))
model = model_OP_fin
print(len(model.layers))

649
649


In [83]:
test_data = my_img_vals[0]
test_data = test_data.reshape(1, img_size[0], img_size[1],3)
test_data_shape = model_OP_fin.predict(test_data).shape
print(test_data_shape)
shape = (my_img_vals.shape[0],test_data_shape[1],test_data_shape[2],test_data_shape[3])
print(shape)
print(shape[1:])

(1, 4, 6, 1664)
(404, 4, 6, 1664)
(4, 6, 1664)


In [84]:
features_ext = []
for i in my_img_vals:
    
    i = i.reshape(1, img_size[0], img_size[1],3)
    features_ext.append(model_OP_fin.predict(i).flatten())
    
features_ext=np.array(features_ext)
print(features_ext.shape)

(404, 39936)


In [85]:
from sklearn.model_selection import train_test_split
from sklearn import svm
y = lables
(X_train, X_test, y_train, y_test) = train_test_split(features_ext, y,test_size=0.1, random_state=1250)

In [86]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [87]:
predicted_y_ = svclassifier.predict(X_test)
print(predicted_y_)
print(y_test)

count = 0
for i in range(len(predicted_y_)):
  if predicted_y_[i] == y_test[i]:
    count = count + 1

print(count / len(predicted_y_))

[12 17 15 38 25  6 19 15 15 24 36 26 30 12 30 29 32  4  5 18 24 15  6 24
 15 15 24 14 38 30 32 23 33 18  0 17 36 32  0  8 20]
[12, 17, 15, 38, 25, 6, 19, 15, 15, 24, 36, 26, 30, 37, 30, 29, 26, 4, 5, 18, 7, 15, 6, 24, 15, 15, 24, 14, 38, 30, 26, 23, 33, 18, 0, 10, 34, 32, 0, 8, 20]
0.8536585365853658


In [88]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predicted_y_))
print(classification_report(y_test,predicted_y_))

[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0 0 0 0]
 [0 0 0 0 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
